In [1]:
import os
# pyspark --packages graphframes:graphframes:0.3.0-spark2.0-s_2.11 --jars graphframes-0.3.0-spark2.0-s_2.11.jar
pyspark_submit_args = '--packages graphframes:graphframes-0.8.0-spark3.0-s_2.12 pyspark-shell'# --jars graphframes-0.8.0-spark3.0-s_2.12.jar'
pyspark_submit_args = '--jars graphframes-0.8.0-spark3.0-s_2.12.jar pyspark-shell'

os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
import findspark
findspark.init()
findspark.find()
import pyspark
conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)

# from pyspark import *


In [2]:
from pyspark import *
from pyspark.sql import *
from graphframes import *
sparkSession = SparkSession.builder.appName('A4').master('local').getOrCreate()
# spark = SparkSession.builder.appName('BDA_A2').master('local').getOrCreate()

In [3]:
dataset = open('soc-Epinions1.txt', 'r')
lines = dataset.readlines()
lines = lines[4:] # skipping the comments in file

In [4]:
vertices = set()
edges = []
for line in lines:
    source, dest = line.split()
    vertices.add((source,))
    vertices.add((dest,))
    edges.append((source, dest))
#     print(source, dest)
#     break
vertices = list(vertices)

In [5]:
vertices_df = sparkSession.createDataFrame(vertices, ["id"])
edges_df = sparkSession.createDataFrame(edges, ["src", "dst"])

In [6]:
graph = GraphFrame(vertices_df, edges_df)

In [8]:
!java --version

Error: Could not create the Java Virtual Machine.
Error: A fatal exception has occurred. Program will exit.
Unrecognized option: --version


In [7]:
graph.inDegrees.show()


+----+--------+
|  id|inDegree|
+----+--------+
|1572|     180|
|2069|      51|
| 296|     121|
|1512|     131|
|2904|      53|
|3414|      16|
| 829|      63|
| 675|      33|
|2088|      80|
|2162|      61|
|4032|      17|
|2294|      56|
|6613|      59|
|6731|       8|
|1436|     181|
| 691|      47|
|9009|      34|
|3210|      34|
|9586|       1|
|9993|       2|
+----+--------+
only showing top 20 rows



In [8]:
results = graph.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

+-----+--------------------+
|   id|            pagerank|
+-----+--------------------+
| 1512|  12.508899597085353|
|15555| 0.18978529864538907|
|25350| 0.03738384133807942|
|62049|  0.1249325425694327|
|15553|0.018785849918632875|
|61959|0.018785849918632875|
|20598| 0.06861505602659947|
|18607| 0.05554855694255348|
|19857|  0.4776621757923465|
|66760|0.051194122772870894|
|19214|  0.2684134323282874|
|61683| 0.02034297723847981|
|22158| 0.09177372448479623|
|30139|0.041249555916683785|
| 6621|    9.14650320024795|
|39229|0.031231704259950383|
|72280|0.018785849918632875|
|10752|0.018785849918632875|
|47797| 0.05401824453325393|
|64349|0.018785849918632875|
+-----+--------------------+
only showing top 20 rows



In [10]:
results.vertices.sort('pagerank' , ascending=False).show()

+----+------------------+
|  id|          pagerank|
+----+------------------+
|  18| 389.0755717978781|
| 737| 353.5986771882279|
|  40|231.13967020425335|
| 118| 220.2080639465201|
| 401|196.23701565826522|
| 725|191.20265292318467|
| 849|185.00521931960506|
| 136|180.81574692246252|
|1719| 180.8121138696468|
| 550| 177.6022990644351|
|  27|176.02101108314434|
| 143|175.17242401681244|
| 135|169.42449507211097|
|  34| 168.4580134620654|
|  77|164.23336900189565|
|1401|164.07834311005158|
|1247|162.10827082315183|
|2227| 158.5934727034573|
| 918|157.46705975143288|
|1191| 156.6757947112163|
+----+------------------+
only showing top 20 rows

